In [1]:
import tensorflow as tf
from tensorflow.keras import backend as K
import cv2
import numpy as np

tf.compat.v1.disable_eager_execution()

In [2]:
# Load model
model = tf.keras.models.load_model("saved_models/checkpoints/artifact-1.06.h5")

Instructions for updating:
Colocations handled automatically by placer.


In [28]:
# Load image
def predict_with_heatmap(image_path):

    I = cv2.imread(image_path)
    I_oryg = I.copy()
    I = cv2.resize(I, (200, 200), interpolation=cv2.INTER_AREA)
    I = cv2.cvtColor(I, cv2.COLOR_BGR2RGB)
    # Predict and print result
    preds = model.predict(np.array([I]))
    print(round(preds[0][0], 2))
    I = cv2.cvtColor(I, cv2.COLOR_RGB2BGR)
    
    # Heatmap below
    x = np.array([I])
    model_output = model.output[:, np.argmax(preds[0])]
    
    last_conv_layer = model.get_layer('block5_conv4')
    grads = tf.gradients(model_output, last_conv_layer.output)[0]
    pooled_grads = K.mean(grads, axis=(0, 1, 2))
    
    iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])
    pooled_grads_value, conv_layer_output_value = iterate([x])
    
    for i in range(512):
        conv_layer_output_value[:, :, i] *= pooled_grads_value[i]
    
    heatmap = np.mean(conv_layer_output_value, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    
    heatmap = cv2.resize(heatmap, (I_oryg.shape[1], I_oryg.shape[0]))
    
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_HOT)
    # Show heatmap on top of image
    
    I_oryg = cv2.cvtColor(I_oryg, cv2.COLOR_BGR2GRAY)
    I_oryg = cv2.cvtColor(I_oryg, cv2.COLOR_GRAY2BGR)
    
    superimposed_img = heatmap * 0.3 + I_oryg * 0.7
    superimposed_img = superimposed_img.astype('uint8')
    
    cv2.namedWindow("output", cv2.WINDOW_NORMAL)
    cv2.imshow("output", superimposed_img)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [73]:
predict_with_heatmap("../../datasets/google_graphics/real/img000041-coco-1036.jpg   ")

0.92
